In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# 데이터 불러오기
### 이미지 불러오는 방법 2가지
알림판을 만들어주기(look up table)
이미지 별로 그 각각 정답을 데이터프레임으로 만들어두기
데이터프레임에 들어가는 2가지 칼럼: 1. 이미지의 경로 2. 정답 레이블



In [ ]:
import os
# list에는 apply를 적용 못함, series로 묶어서 apply로 적용함
train = pd.DataFrame({ 
                      "path" :pd.Series(os.listdir("../input/train")).apply(lambda x : "../input/train/" + str(x)), 
                      "label" :pd.Series(os.listdir("../input/train")).apply(lambda x : str(x).split(".")[0])})
train.head()

In [ ]:
# 이미지를 배치사이즈로 처리하겠다. 이미지들을 덩어리별로 처리하겠다 (한번에 이미지들을 전부 다 넣어버리면 데이터가 너무커서 모델이 안돌아감)
# 배치로 굉장히 쉽게 불러오는 방법이 존재한다: ImageDataGenerator 라이브러리

from keras.preprocessing.image import ImageDataGenerator

# 모델을 선언한다.  옵션 필수적으로 2개 알기: 
#거리기반 학습해주는 모델의 경우 정교화 필수!
#rescale: 픽셀값(0~255)을 정규화시켜준다.(모델의 학습속도가 빨라진다), 데이터 전처리를 할때 각 픽셀들의 숫자를 정규화 해준다! 그러면 범위가 0~1로 줄어들어서 모델 학습에 큰 도움을 준다
#validation_split: validation(평가set)양 설정, 전체데이터셋중에 20%의 데이터를 평가셋으로 , overfitting을 막기위해서,  epochs(학습)을 거칠때마다 val_loss 확인 가능
train_image_gen = ImageDataGenerator(rescale=1/255, validation_split=0.2)

#flow_from_dataframe 함수 : 데이터프레임으로 부터 데이터를 가져온다   /  flow_from_directory() << 너무쉬운버전 
#dataframe:들어갈 데이터셋 , target_size: 모델에 넣기전에 사진 사이즈를 일정하게 해줘야한다. 모델은 훈련할때 똑같은 이미지 사이즈만 받아야함 subset: 어디에 쓰이는 데이터인지 설정~ training
train_generator = train_image_gen.flow_from_dataframe(dataframe = train, x_col = "path", y_col ="label",\
                                                      target_size=(75,75), subset='training')


In [ ]:
#validation generator를 만든다.
val_generator = train_image_gen.flow_from_dataframe(dataframe = train, x_col = "path", y_col ="label", \
                                                    target_size=(75,75), subset='validation')

In [ ]:
# 모델을 불러온다
from keras.layers import Dense #층
from keras.applications.inception_v3 import InceptionV3 #inceptionV3 라는 딥러닝 모델
from keras.models import Model #딥러닝 쌓는 방법 2가지(sequential, functional)중 한가지 이용 ~ sequential: 층층이 쌓는 방식,단점: 옆으로 안쌓고 그냥 직선으로만 쌓음/ functional: input층 , output층 명시해줘야함
# 여기서는 functional model을 이용한다

# 모델을 선언한다
# inceptionV3 라는 딥러닝 모델선언: 자동으로 layer들 쌓아둠(학자들이 만들어둔 모델), y = Wx + b(가중치 W를 찾아라~ 이게 딥러닝 모델 학습이유)
# Pretrain된 모델 인셉션 V3: 이미 W 가중치가 어느정도 훈련된 모델, imagenet에서 이미 수많은 데이터를 이미 학습해둠
# include_top:  출력층인 맨마지막 층은 안 가져오겠다.(imagenet은 1000개 클래쓰 나누는 데이터를 학습했다 . 1000개 클래쓰를 나누는 층을 가져올 것이 아니다 왜냐면 여기서는 2개(dog,cat)의 클래쓰만 나누는 작업이므로)
# pooling: average_pooling, max_pooling이 있음(둘중에 하나씀) ~ 1) 차원축소의 효과, 2) 과적합도 막아줄 수 있음
InceptionV3_base_model = InceptionV3(weights='imagenet', include_top=False, pooling = "avg") #많은 층들이 쌓여있음

x = InceptionV3_base_model.output # inceptionv3의 출력층을 가져온다 #functional api를 이용한 층쌓기이기 때문에 꼭 해줘야하는 작업



In [ ]:
InceptionV3_base_model.summary()  #층을 보여줌(print)

In [ ]:
InceptionV3_base_model.output

In [ ]:
# Dense layer로 출력층 결정, Dense layer는 히든 layer 층으로도 출력 layer층으로도 쓰일 수 있음, 이미지 처리 할때는 출력 layer층으로 많이 쓰임

# 숫자 2~ 클래쓰 2개 분류,   inception 출력층 x를 인풋으로 
#  숫자-> 히든 노드의 갯수, activation -> 활성화 함수 (특히 ReLU가 성능이 좋음) , softmax 쓴 이유는 틀렸을떄 패널티를 너무 팍팍주지 않기 위해서,다중 분류 문제면 softmax  / 이진 분류면 softmax, sigmoid 
#딥러닝 모델은 각 층마다 W 값이 존재한다 
final_pred = Dense(2, activation='softmax')(x)  #functional api는 x라는 모델의 출력층을 입력층으로 붙여줘야함  그래서 바로 뒤에 (x) 라고 표시해줌
final_pred #최종 출력층

In [ ]:
#모델의 인풋 층
InceptionV3_base_model.input

In [ ]:
# model = Model(inputs=인풋 층, outputs=아웃풋 층)
model = Model(inputs=InceptionV3_base_model.input, outputs=final_pred)

In [ ]:
# Compile 과정
# model.compile(옵티마이저 = , 로스 =,) optimizer는 기본적으로 adam을 쓰면 됨, adam: 그레디언트 디센트 방법론 이용할때 어떤 방향으로 내려갈지 결정(loss를 제일 줄이는 방향으로)  adam이 성능 안나오면 SGD를 고려하면 됨
# loss: 우리 모델이 학습을 잘하고 있는 지 없는지를 loss를 보고 평가 , 다중 분류 문제를 풀 때는 categorical_crossentropy, 이진분류 문제풀때는 바이너리 크로스엔트로피, 회귀문제(양을 예측할때)를 풀때는 mean_squared_error
# 엔트로피(무질서도)가 낮아지는 방향으로 학습 해야한다
# 다중 분류 문제를 풀거나 분류를 풀때! 대표적인 모델의 평가방식은 Log Loss(=categorical_crossentropy)
model.compile(optimizer="adam", loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# 학습하기 
# 딥러닝은 model.fit_generator()  사용, batch size를 처리하기 위해 fit이 아닌 fit_generator를 사용
# train_generator에 이미 y값이 들어가 있기 때문에 그냥 이거 하나만 넣어도 됨
# 딥러닝에서 전체데이터셋을 한번 쭉~ 학습을 하는것을 epochs 한번 거쳤다고한다
# 400 ~ 가중치가 몇번 수정 될것인가    훈련데이터사이즈(2만개) // 배치사이즈(50) = 400   , 배치사이즈: 한번 전체데이터셋을 학습할때 거치는 스텝의 수
# steps_per_epoch = 훈련 데이터 사이즈 // 배치 사이즈
# validation_steps = 평가셋 데이터 사이즈 // 배치 사이즈
# val_loss가 낮아지는 것이 좋은 것이다
model.fit_generator(train_generator, epochs=3,
                                                steps_per_epoch=20000//50,
                                                validation_data=val_generator,
                                                validation_steps=5000//50)

In [ ]:
# 테스트셋 평가

test = pd.DataFrame({
                      "path" :pd.Series(os.listdir("../input/test")).apply(lambda x : "../input/test/" + str(x)), 
                     })

test_image_gen = ImageDataGenerator(rescale=1/255)

#배치 사이즈는 꼭 train꺼랑 꼭 맞혀줄 필요는 없다
#class_mode => 다중 분류 문제 일때는 None이라고 해주면 됨
#shuffle => test 셋에서는 FALSE로 해줘야함,    train set에서는 True 해줘야 섞어줘서 훈련이 잘 됨   
test_generator = test_image_gen.flow_from_dataframe(dataframe = test, x_col = "path", y_col = None,\
                                                    target_size=(75,75), batch_size=50, class_mode=None, shuffle=False)


#50 : 배치 사이
#np.ceil => 숫자를 올림함으로써, 오류 없이 동작하도록
y_pred = model.predict_generator(generator=test_generator, steps=int(np.ceil(len(test)/50))-1)

In [ ]:
y_pred # 왼쪽이 고양이 오른쪽 강아지

In [ ]:
# 제출 코드: 강아지냐 아니냐
y_pred[:, 1]

In [ ]:
test_generator.filenames

In [ ]:
# 제출
# 대회측 요구 칼럼: id, label
# id => test_generator.filenames 이미지들이 들어가 있음
# label=> 강아지일 확률을 넣어줌
submission = pd.DataFrame({'id':pd.Series(test_generator.filenames),'label':pd.Series(y_pred[:,1])})

In [ ]:
submission.head()

In [ ]:
submission['id'] = submission["id"].apply(lambda x: str(x).split("/")[3].split(".")[0]).astype("int")

In [ ]:
submission.to_csv('DogVsCats_submission.csv',index=False)